# 半监督算法

半监督学习适用于样本标签数据不全的情况,好比有1000组样本,其中只有100个有标签其他都没有标签,那么就可以适当的使用半监督算法.半监督算法一般多是一些图模型.通过一些先验的参数来捕获数据的底层分布形状,用以推广到新的未标记样本之上.

sklearn中提供了两种非常基础的半监督学习算法:

+ LabelPropagation 标签传递法
+ LabelSpreading 标签扩散法

这两个算法都是通过构建相似图来推断标签的.

`LabelPropagation` 和 `LabelSpreading`在对图形的相似性矩阵,以及对标签分布的夹持效应的修改方面不太一样.Clamping允许算法在一定程度上改变真实标签化数据的权重.LabelPropagation算法执行输入标签的hard clamping, 这意味着 $\alpha=0$.这些 clamping factor 可以不是很严格的,例如 $\alpha=0.2$意味着我们将始终保留原始标签分配的 80%, 但该算法可以将其分布的置信度改变控制在 20% 以内.

`LabelPropagation`使用原始相似性矩阵从未修改的数据来构建. LabelSpreading 通过最小化具有正则化属性的损耗函数实现, 因此它通常更适用于有噪音的数据.该算法在原始图形的修改版本上进行迭代,并通过计算归一化图拉普拉斯矩阵(normalized graph Laplacian matrix)来对边缘的权重进行归一化.此过程也用于`Spectral clustering`

这两种标签传播模型有两种内置的核函数,其选择会影响算法的可扩展性和性能:

+ rbf $(\exp(-\gamma |x-y|^2), \gamma > 0)$. $\gamma$ 通过关键字 gamma 来指定.

    RBF 核将产生一个完全连接的图形,它通过密集矩阵在内存中表示.该矩阵可能非常大,与算法的每次迭代执行全矩阵乘法计算的成本相结合可导致超长的运行时间.

+ knn $(1[x' \in kNN(x)])$. k 通过关键字 n_neighbors 来指定

    KNN 核将产生更多的内存友好的稀疏矩阵,这样可以大幅度的减少运行时间.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading

In [2]:
label_prop_model = LabelSpreading()
iris = datasets.load_iris()
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(len(iris.target)) < 0.3
labels = np.copy(iris.target)
labels[random_unlabeled_points] = -1# 位置标签记为-1
label_prop_model.fit(iris.data, labels)

LabelSpreading(alpha=0.2, gamma=20, kernel='rbf', max_iter=30, n_jobs=1,
        n_neighbors=7, tol=0.001)

半监督学习是个很宽也很深的领域,本文只是基于sklearn介绍了两个算法而已,这两个方法其实也早已不在前沿.更多的半监督学习的进展可以多关注论文,比如[这篇Improved GAN](https://arxiv.org/pdf/1606.03498.pdf)